# Read Varian DVH Text Files

## Imports

In [2]:
from typing import Callable, List, Any, Dict
from pathlib import Path
import re
from functools import partial

import numpy as np
import pandas as pd

import text_reader as tp
from buffered_iterator import BufferedIterator
from sections import Rule, RuleSet, SectionBreak, Section, ProcessingMethods


## Demo File Path

In [3]:
demo_dvh_folder = Path.cwd() / r'./References/Text Files/DVH files'
demo_dvh_1 = demo_dvh_folder / 'Breast CHWR Relative Dose Relative Volume 1 cGy Step Size.dvh'
#demo_dvh_1.exists()
#demo_dvh_folder.exists()
multi_dvh = demo_dvh_folder / 'Replan1, Replan2, Replan3 Comparison DVH Absolute Dose Relative Volume 1 cGy Step Size.dvh'

## DVH File Sections
<style type="text/css">
.tg  {text-align:center;font-family:Arial, sans-serif;font-size:14px;font-weight:bold;}
.th{text-align:center;background-color:#38fff8;font-weight:bold;}
</style>
<table>
<thead><tr><th class="th">Section</th><th class="th">Description</th></tr></thead>
<tbody>
    <tr>
        <td>Patient Information</td>
        <td>
            First Section in file.<br>
            Occurs only Once.<br>
        </tr>
    <tr>
        <td>DVH Information</td>
        <td>
            Information about the DVH source and format.<br>
            Occurs only Once.
            </td>
        </tr>
    <tr>
        <td>Plan Information</td>
        <td>
            Information about the plan generating the DVH.<br>
            Repeated for each plan included in the DVH file.
            </td>
        </tr>
    <tr>
        <td>Structure Information</td>
        <td>
            Information about each structure in the plan DVH.<br>
            Repeated for each Structure included in the DVH file.
            </td>
        </tr>
    <tr>
        <td>Structure Dose Summary</td>
        <td>
            A Summary of the DVH Dose information for the structure.<br>
            Repeated for each Structure included in the DVH file.
            </td>
        </tr>
    <tr>
        <td>DVH Data</td>
        <td>
            Columns containing DVH data for the structure.<br>
            Occurs immediately after the related Structure Information 
            for each Structure included in the DVH file.      
            </td>
        </tr>
</tbody>
</table>

## Initial Header
The initial header contains two parts:
- Patient Information
- DVH Information
  
The information is formatted as two fixed-width columns
- The first column is left-justified with trailing spaces and a final _':'_. 
- The second column is also left-justified but without trailing spaces.
- The second column begins with a space.
- The _'Description'_ value extends over multiple lines with spaces filling the entire first column.
- A blank line occurs at the end of the initial header

<style type="text/css">
    .tg {text-align:left;background-color:#e6ffff;font-family:Arial, sans-serif;font-size:14px;font-weight:bold;border-style: solid dotted solid dotted}
    .th {text-align:center;background-color:#38fff8;font-weight:bold;}
    .tc {text-align:left;background-color:#e0ebeb;font-family: Terminal, monospace;font-size:11px;font-weight:normal;}
</style>
<table>
    <thead><tr>
        <th class="th">Export File Entry</th>
        <th class="th">Value</th>
        <th class="th">Example</th>
    </tr></thead>
<tbody>
    <tr><td colspan="3" class="tg">Patient Information</td></tr>
    <tr>
        <td>Patient Name:</td>
        <td>Name of the patient</td>
        <td class="tc">AXIR, CHWR</td>
        </tr>
    <tr>
        <td>Patient ID:</td>
        <td>Identification of the patient</td>
        <td class="tc">TEST CHWR</td>
        </tr>
    <tr><td colspan="3" class="tg">DVH Information</td></tr>
        <tr>
            <td>Comment:</td>
            <td>User defined comment</td>
            <td class="tc">DVHs for a plan sum</td>
            </tr>
        <tr>
            <td>Date:</td>
            <td>Date and time as defined in the Windows operating system</td>
            <td class="tc">March 17, 2023 11:40:38 AM</td>
            </tr>
        <tr>
            <td>Exported by:</td>
            <td>User name of the user who exported the DVH</td>
            <td class="tc">GS MP</td>
            </tr>
        <tr>
            <td>Type:</td>
            <td>Type of the DVH One of:<br>
                Cumulative<br>
                Differential<br>
                Natural<br>
                </td>
            <td class="tc">Cumulative Dose Volume Histogram</td>
            </tr>
        <tr>
            <td>Description:</td>
            <td>Description of the DVH type exported</td>
            <td class="tc">
                The cumulative DVH displays the percentage (relative)<br>
                or volume (absolute) of structures that receive a dose<br>
                equal to or greater than a given dose.<br>
                </td>
            </tr>
</tbody>
</table>

__Example Initial Header__ (Numbers at the top are for reference only)

> ```
>00000000001111111111222222222233333333334444444444555555555566666666667777777777
>01234567890123456789012345678901234567890123456789012345678901234567890123456789
>
>Patient Name         : AXIR, CHWR
>Patient ID           : TEST CHWR
>Comment              : DVHs for a plan sum
>Date                 : March 17, 2023 11:40:38 AM
>Exported by          : GS MP
>Type                 : Cumulative Dose Volume Histogram
>Description          : The cumulative DVH displays the percentage (relative)
>                       or volume (absolute) of structures that receive a dose
>                       equal to or greater than a given dose.
>
> ```

### Information Section Parameters
**Notes**
- Combine *Patient Information* and *DVH Information* into one section.
- Drop the *Description* since it doesn't contain useful information and is 
  more complicated to deal with because it spans multiple lines. 
- Split each line at the first ':'
- Remove leading and trailing spaces from each part of the split
- Convert the split rows into dictionary items, with the first item as 
  the key and the second as the value, dropping any empty rows.
  
**Parameters**
<table>
<tr><td>name</td><td>Information</td><td>Section name for reference</td></tr>
<tr><td>start_section</td><td>None</td><td>Beginning of file</td></tr>
<tr><td>start_search</td><td>False</td><td>Start section immediately</td></tr>
<tr><td>end_section</td><td>('Description', 'START', 'Before')</td>
  <td>End the section just before the line that begins with 'Description'</td>
  </tr>
<tr><td>processor</td>
  <td>[partial(str.split, sep=':', maxsplit=1),<br>
       tp.trim_items,<br>
       tp.drop_blanks]</td>
  <td>Split each line at the first ':',<br> 
      remove leading and trailing spaces from each part of the split,<br>
      then drop any empty rows.</td>
  </tr>
  <tr><td>assemble</td><td>tp.to_dict</td>
  <td>Convert the split rows into dictionary items, with the first item as 
      the key and the second as the value.</td>
  </tr>
  </table>

In [4]:
info_split = partial(str.split, sep=':', maxsplit=1)
dvh_info_section = Section(
    name='Information',
    start_section=None,
    end_section=('Description', 'START', 'Before'),
    processor=[info_split, 
               tp.trim_items, 
               tp.drop_blanks],
    assemble=tp.to_dict
    )

In [5]:
demo_dvh_text = demo_dvh_1.read_text(encoding='utf_8_sig').splitlines()
dvh_info_section.read(demo_dvh_text)

{'Patient Name': 'AXIR, CHWR',
 'Patient ID': 'TEST CHWR',
 'Comment': 'DVHs for one plan',
 'Date': 'March 17, 2023 11:40:38 AM',
 'Exported by': 'GS MP',
 'Type': 'Cumulative Dose Volume Histogram'}

## Plan Information
- _Plan Information_ occurs immediately after the _Initial Header_.
- _Plan Information_ is repeated for each plan in the DVH file.
   Often there will be only one plan, but DVH files generated from plan 
   comparison DVHs will contain multiple plans.
- The `Plan` and `Course` information in the _Plan Information_ is also 
  contained in each _Structure Information_ section, and can be used to link the 
  `Total dose` to the appropriate _DVH Dose Data_.
- `Total dose` may be important if conversions between absolute and relative 
  dose are required.
- The Plan Header information is formatted as delimited text, with ': ' as 
  the delimiter.
- For a _Plan sum_ DVH `Total dose [cGy]` and `% for dose (%)` will have 
  values of: `not defined`
- A blank line occurs at the end of the plan header

<style type="text/css">
    .tg {text-align:left;background-color:#e6ffff;
        font-family:Arial, sans-serif;font-size:14px;font-weight:bold;
        border-style: solid dotted solid dotted}
    .th {text-align:center;background-color:#38fff8;font-weight:bold;}
    .tc {text-align:left;background-color:#e0ebeb;
        font-family: Terminal, monospace;font-size:11px;font-weight:normal;}
</style>
<table>
<thead><tr>
    <th class="th">Export File Entry</th>
    <th class="th">Value</th>
    <th class="th">Example</th>
    </tr></thead>
<tbody>
    <tr>
        <td>Plan: <b>OR</b><br>Plan sum:</td>
        <td>Plan ID</td>
        <td class="tc">Plan sum: Plan Sum</td>
        </tr>
    <tr>
        <td>Uncertainty plan:</td>
        <td>Plan ID (variation of plan: Plan ID)<br>
            Not present if Plan Uncertainty is not calculated.
            </td>
        <td class="tc">N/A</td>
        </tr>
    <tr>
        <td>Course:</td>
        <td>Course ID</td>
        <td class="tc">Course: C1</td>
        </tr>
    <tr>
        <td>Plan Status:</td>
        <td>One of:
            <ul>
                <li>Planning Approved</li>
                <li>Treatment Approved</li>
                <li>Reviewed</li>
                <li>Unapproved</li>
                <li>Rejected</li>
                <li>Completed</li>
                </ul>
            Not present if the DVH is for a <i>Plan sum</i>.
            </td>
        <td class="tc">
            Plan Status: Treatment Approved 
            Thursday, January 02, 2020 12:55:56 
            by gsal
            </td>
        </tr>
    <tr>
        <td>Total dose [Gy]:</td>
        <td>Total dose in specified units</td>
        <td class="tc">Total dose [cGy]: 6400.0</td>
        </tr>
    <tr>
        <td>% for dose (%):</td>
        <td>Treatment prescription percentage</td>
        <td class="tc">% for dose (%): 100.0</td>
        </tr>
</tbody>
</table>

__Example Plan Header__

> ```
> Plan: EARR
> Course: C2
> Plan Status: Completed
> Total dose [cGy]: 6400.0
> 
> ```

### Plan Information Notes

- Start the section with a line that begins with *Plan:* **OR** *Plan sum:*
- End after the line containing *% for dose (%)* 
- Split each line at the first ':'
- Remove leading and trailing spaces from each part of the split
- Convert the split rows into dictionary items, with the first item as 
  the key and the second as the value, dropping any empty rows.
- If *Plan sum*, convert key to *Plan*
- If *Plan sum*, Add context information indicating that it is a plan sum
- If *Plan Status* contains approval information, split this information into 
  three parts.  e.g.<br>
  `Treatment Approved Thursday, January 02, 2020 12:55:56 by gsal` becomes:
    - 'Plan Status': 'Treatment Approved'
    - 'Approved on': 'January 02, 2020 12:55:56'
    - 'Approved by':  'gsal'
- Extract the prescribe dose units as a separate item and convert the dose to 
  a number e.g.<br>
    `Total dose [cGy]: 6400.0` becomes: 
    - 'Prescribed dose': 6400.0
    - 'Prescribed dose unit': 'cGy'
- Convert the prescription isodose line to a number e.g.<br>
  `% for dose (%): 100.0` becomes: 
    - 'Prescribed isodose line': 100.0
- If *Total dose* or  *% for dose (%)* have values of: `not defined`, replace 
  it with an empty string.


### Custom Processing Functions
The Plan sections requires custom functions to reliably extract information in 
a convenient format. The custom functions are defined as **Rules**, which 
identify the appropriate line(s) and then apply custom formatting.

Since the desired assembled result for the section is a dictionary, and since 
the dictionary is conveniently created from a sequence of length-two lists, 
each rule returns (or yields) one or more length-two lists.

The **Rules** are then combined as a **RuleSet**, where the rules are applied 
in order until one of them passes and the result of that rule is returned.

A default Processing Function is given that is applied if none of the rules 
pass.  This default function also returns a length-two list by splitting the 
line at the first ':' and returning the two parts after stripping spaces.

In [6]:
def plan_split(line: str)->List[str]:
    '''Spilt a text line into two parts on ':'.

    Spilt a text line into two parts on the first occurrence of ':'.
    Remove leading and trailing spaces from each part.
    Force the returned list to have length of two even if the text does not 
    contain a ':'.

    Args:
        line (str): The test to spilt

    Returns:
        List[str]: A length-2 list of strings
    '''
    parts = line.split(sep=':', maxsplit=1)
    # Remove leading and trailing spaces from each part
    clean_parts = [s.strip() for s in parts]
    # Force clean_parts to be a length of 2
    if len(clean_parts) == 1:
        clean_parts.append('')
    return clean_parts

#### Approved Status Rule

- Identify lines containing the text *'Treatment Approved'*
- Convert a line like:<br>
   `Plan Status: Treatment Approved Thursday, January 02, 2020 12:55:56 by gsal`<br>
   into into 3 lines:
    1. ['Plan Status', 'Treatment Approved']
    2. ['Approved on', Thursday, January 02, 2020 12:55:56]
    3. ['Approved by', gsal]
- One-by-one yield each two-item list as if each one were a separate line.

- The approval date is the text between *'Treatment Approved'* and *' by'*.
- The user is the text after *' by'*.
             

Uses the following regular expression:
> `.*(?P<approval>Treatment Approved)\s*(?P<date>.*)\s*by\s*(?P<user>.*?)\s*$`

|Portion of regular expression|Meaning                                |
|-----------------------------|---------------------------------------|
|.*                           |Initial text                           |
|(?P<approval>                |Beginning of approval capture group    |
|Treatment Approved           |Literal text 'Treatment Approved'      |
|)                            |End of approval capture group          |    
|\s*                          |Possible whitespace                    |
|(?P<date>.*)                 |Text containing approval date          |
|\s*                          |Possible whitespace                    |
|by                           |Literal text 'by'                      |
|\s*                          |Possible whitespace                    |
|(?P<user>.*?)                |Text containing user (non-greedy)      |
|\s*                          |Possible trailing whitespace           |
|$                            |end of string                          |

In [7]:
def make_approved_status_rule() -> Rule:
    '''If Treatment Approved, Split "Plan Status" into 3 lines.

    Accepts a supplied line like:
    `Plan Status: Treatment Approved Thursday, January 02, 2020 12:55:56 by gsal`,
    Extracts and user.
    The approval date is the text between event.test_value and ' by'.
    The user is the text after ' by'.
    Yields three two-item lists.   
    A supplied line like:
    `Plan Status: Treatment Approved Thursday, January 02, 2020 12:55:56 by gsal`,
    Gives:
        [['Plan Status', 'Treatment Approved'],
            ['Approved on', Thursday, January 02, 2020 12:55:56],
            ['Approved by', gsal]
    '''    
    def approved_status_parse(line, event) -> tp.ProcessedList:
        match_results = event.test_value.groupdict()
        parsed_lines = [
            ['Plan Status', match_results['approval']],
            ['Approved on', match_results['date']],
            ['Approved by', match_results['user']]
            ]
        for line in parsed_lines:
            yield line

    approval_pattern = (
        r'.*'                  # Initial text
        r'(?P<approval>'       # Beginning of approval capture group
        r'Treatment Approved'  # Literal text 'Treatment Approved'
        r')'                   # End of approval capture group        
        r'\s*'                 # Possible whitespace
        r'(?P<date>.*)'        # Text containing approval date
        r'\s*'                 # Possible whitespace
        r'by'                  # Literal text 'by'
        r'\s*'                 # Possible whitespace
        r'(?P<user>.*?)'       # Text containing user (non-greedy)
        r'\s*'                 # Possible trailing whitespace
        r'$'                   # end of string
        )
    re_pattern = re.compile(approval_pattern)
    approved_status_rule = Rule(name='approved_status_rule',
                                sentinel=re_pattern, 
                                pass_method= approved_status_parse, 
                                fail_method='None')
    return approved_status_rule

#### Prescribed Dose Rule

- Split lines with Prescribed or Total dose into 2 lists with dose value and 
  dose unit.
- Identify lines with the format:<br>
    > Total dose [*unit*]: *dose*  ***OR***<br>
    > Prescribed dose [*unit*]: *dose*    
- Yield 2 lists:
    1. ['Prescribed dose', *dose*],
    2. ['Prescribed dose unit', *unit*]
- If the *dose* is the text 'not defined' Use `np.nan` for the dose and and 
  an empty string for the unit.
    > The line:<br>
    > `'Prescribed dose [cGy]: 5000.0'`<br>
    > Results in:<br>
    > `['Prescribed dose', 5000.0],`<br>
    > `['Prescribed dose unit', 'cGy']`
    
    > The line:<br>
    > `'Total dose [cGy]: not defined'`<br>
    > Results in:<br>
    > `['Prescribed dose', np.nan],`<br>
    > `['Prescribed dose unit', '']`
    
- One-by-one yield each two-item list as if each one were a separate line.


Uses the following regular expression:
> `^(Total|Prescribed)\s*dose\s*\[(?P<unit>[A-Za-z]+)\]\s*:\s*(?P<dose>[0-9.]+|not defined)\s*$`

|Portion of regular expression|Meaning                                      |
|-----------------------------|---------------------------------------------|
|^(Total|Prescribed)  |String begins with 'Total' OR 'Prescribed'           |
|\s*dose\s*           |Literal text 'dose' surrounded by possible whitespace|
|\[                   |Unit start delimiter '['                             |    
|(?P<unit>[A-Za-z]+)  |unit group: text surrounded by []                    |
|\]                   |Unit end delimiter ']'                               |
|\s*:\s*              |Dose delimiter ':' surrounded by possible whitespace |
|(?P<dose>            | Beginning of dose group                             |
|[0-9.]+              |dose group Number                                    |
||not defined         |"not defined" alternative to number                  |
|)                    | End of dose group                                   |
|[\s\r\n]*            |Possible trailing whitespace                         |
|$                    |end of string                                        |


In [8]:
# Prescribed Dose Rule
def make_prescribed_dose_rule() -> Rule:
    '''Split Dose into dose vale and dose unit.
    For a line containing:
        Total dose [unit]: dose  OR
        Prescribed dose [unit]: dose
    The line:
        Prescribed dose [cGy]: 5000.0
    Results in:
        ['Prescribed dose', '5000.0'],
        ['Prescribed dose unit', 'cGy']
    The line:
        Total dose [cGy]: not defined
    Results in:
        ['Prescribed dose', ''],
        ['Prescribed dose unit', '']
    '''
    def parse_prescribed_dose(line, event) -> tp.ProcessedList:
        match_results = event.test_value.groupdict()
        # Convert numerical dose value to float and 
        # 'not defined' dose value to np.nan
        if match_results['dose'] == 'not defined':
            match_results['dose'] = np.nan
            match_results['unit'] = ''
        else:
            match_results['dose'] = float(match_results['dose'])

        parsed_lines = [
            ['Prescribed dose', match_results['dose']],
            ['Prescribed dose unit', match_results['unit']]
            ]
        for line in parsed_lines:
            yield line

    prescribed_dose_pattern = (
        r'^(Total|Prescribed)'  # Begins with 'Total' OR 'Prescribed'
        r'\s*dose\s*'           # Literal text 'dose' surrounded by whitespace
        r'\['                   # Unit start delimiter '['
        r'(?P<unit>[A-Za-z]+)'  # unit group: text surrounded by []
        r'\]'                   # Unit end delimiter ']'
        r'\s*:\s*'              # Dose delimiter with possible whitespace
        r'(?P<dose>[0-9.]+'     # dose group Number
        r'|not defined)'        #"not defined" alternative
        r'[\s\r\n]*'            # drop trailing whitespace
        r'$'                    # end of string
        )
    re_pattern = re.compile(prescribed_dose_pattern)
    dose_rule = Rule(sentinel=re_pattern, name='prescribed_dose_rule',
                        pass_method= parse_prescribed_dose, fail_method='None')
    return dose_rule

#### Prescribed Isodose Line Rule

- Split lines with Prescribed or Total dose into 2 lists with dose value and 
  dose unit.
- Identify lines with the format:<br>
    > '% for dose (%): *isodose*
- Convert *isodose* to a float value
- Return a list:
    > ['Prescription Isodose', *float isodose*]

In [9]:
# Prescribed Isodose Line Rule
def make_prescribed_isodose_rule() -> Rule:
    '''Identify Prescribed isodose text lines. and convert them into a
    two-item list, with the isodose percentage converted to a number.

    For a line containing '% for dose (%): 100.0':
    Return:
        ['Prescription Isodose', 100.0]
    '''
    def parse_isodose(line, event) -> tp.ProcessedList:
        # Split the line at ':'
        parts = line.split(':')
        isodose_text = parts[1].strip()
        isodose = float(isodose_text)
        parsed_line = ['Prescription Isodose', isodose]
        return parsed_line
    prescribed_isodose_rule = Rule('% for dose (%)', location='IN',
                                   pass_method=parse_isodose,
                                   fail_method='None',
                                   name='make_prescribed_isodose_rule')
    return prescribed_isodose_rule

#### Plan Sum Rule
- If a line starts with 'Plan sum' Convert the text to 'Plan'


In [10]:
# Plan Sum Rule
def make_plan_sum_rule() -> Rule:
    '''Identify lines starting with Plan sum and convert them into a two-item 
    list, with the first item being 'Plan' and the second item being the text 
    after the ':'.
    '''
    def parse_plan_sum(line, event) -> tp.ProcessedList:
        # Split the line at ':'
        parts = line.split(':', maxsplit=1)
        plan_sum_id = parts[1].strip()
        parsed_line = ['Plan', plan_sum_id]
        return parsed_line
    plan_sum_rule = Rule('Plan sum', location='START',
                         pass_method=parse_plan_sum,
                         fail_method='None',
                         name='make_plan_sum_rule')
    return plan_sum_rule

#### Build the RuleSet from the above Rules

In [11]:
plan_rule_set = RuleSet([make_approved_status_rule(),
                         make_prescribed_dose_rule(),
                         make_prescribed_isodose_rule(),
                         make_plan_sum_rule()],
                        default=plan_split)

### Plan Section Parameters
<table>
<tr><td>name</td><td>Plan</td><td>Section name for reference</td></tr>
<tr><td>start_section</td><td>(['Plan:', 'Plan sum:'], 'START', 'Before')</td>
    <td>Start the section just before the line that begins with 
        <i>Plan:</i> <b>OR</b> <i>Plan sum:</i>
        </td>
    </tr>
<tr><td>end_section</td><td>('% for dose (%)', 'START', 'After')</td>
    <td>End the section after the line that begins with '% for dose (%)'</td>
    </tr>
<tr><td>processor</td>
  <td>[plan_rule_set]</td>
  <td>A set of sectionary Rules that result in each line being split into 
      two-item lists</td>
  </tr>
  <tr><td>assemble</td><td>tp.to_dict</td>
  <td>Convert the split rows into dictionary items, with the first item as 
      the key and the second as the value.</td>
  </tr>
  </table>

In [12]:
plan_info_section = Section(
    name='Plan',
    start_section=(['Plan:', 'Plan sum:'], 'START', 'Before'),
    end_section=('% for dose (%)', 'START', 'After'),
    processor=[plan_rule_set],
    assemble=tp.to_dict
    )

In [13]:
demo_dvh_text = demo_dvh_1.read_text(encoding='utf_8_sig').splitlines()
plan_info_section.read(demo_dvh_text)

{'Plan': 'CHWR',
 'Course': 'C1',
 'Plan Status': 'Completed',
 'Prescribed dose': 4250.0,
 'Prescribed dose unit': 'cGy',
 'Prescription Isodose': 100.0}

## Combined Plans Section
This section reads through all *Plan* sections in the text and combines them into a lookup table that can be used with the dose data.

### Plan Lookup Table
- Combine the dictionaries from each *Plan* section into a DataFrame.
- Set 'Course' and 'Plan' IDs as the index so that prescription data can be obtained.

In [14]:
def plan_lookup(plan_sections: List[Dict[str, Any]], 
                context: Dict[str, Any])->Dict[str, Dict[str, Any]]:
    '''Build a dictionary of plan information and add it to context.
    '''
    all_plans = pd.DataFrame(plan for plan in plan_sections if plan)
    all_plans.set_index(['Course', 'Plan'], inplace=True)
    context['PlanLookup'] = all_plans
    return all_plans

### All_Plan Section Parameters
<table>
<tr><td>name</td><td>Plan</td><td>Section name for reference</td></tr>
<tr><td>start_section</td><td>(['Plan:', 'Plan sum:'], 'START', 'Before')</td>
    <td>Start the section just before the first line that begins with 
        <i>Plan:</i> <b>OR</b> <i>Plan sum:</i>
        </td>
    </tr>
<tr><td>end_section</td><td>('Structure', 'START', 'Before')</td>
    <td>End the section before the first line that begins with 'Structure'<br>
       'Structure' marks the start of the DVH Dose Data and the end of all 
       *Plan* sections
       </td>
    </tr>
<tr><td>processor</td>
  <td>[plan_info_section]</td>
  <td>Repeatedly apply the plan_info_section to the text, collecting
      dictionaries of Plan data from each.
      </td>
  </tr>
  <tr><td>assemble</td><td>plan_lookup</td>
  <td>Combine the dictionaries into a DataFrame and set the index so that it
      can be used as a lookup table to obtain prescription data.<br>
      Add the lookup table to context, so that it is accessible to Dose Data
      sections later.
      </td>
  </tr>
  </table>

In [15]:
multi_dvh_text = multi_dvh.read_text(encoding='utf_8_sig').splitlines()
all_plans = Section(
    name='All Plans',
    start_section=(['Plan:', 'Plan sum:'], 'START', 'Before'),
    end_section=('Structure', 'START', 'Before'),
    processor=[plan_info_section],
    assemble=plan_lookup
    )
context = {}
plan_lookup = all_plans.read(multi_dvh_text, context=context)
context['PlanLookup']

Plan Status  Prescribed dose Prescribed dose unit  \
Course      Plan                                                              
NOT TREATED ORBR2-Ref4Gy     Rejected           2000.0                  cGy   
            ORBR-0.5cmBol    Rejected           2000.0                  cGy   
            ORBR3-ref2cm     Rejected           2000.0                  cGy   

                           Prescription Isodose  
Course      Plan                                 
NOT TREATED ORBR2-Ref4Gy                  100.0  
            ORBR-0.5cmBol                 100.0  
            ORBR3-ref2cm                  100.0

## DVH Dose Data
DVH Dose Data is repeated for each structure in the plan that has a calculated DVH.

Each section of DVH data consists of three parts:
- Structure Information
- Structure Dose Summary
- DVH Curve Data


### _Structure Information_ and _Structure Dose Summary_
- _Structure Information_ and _Structure Dose Summary_ are both formatted as 
  delimited text, with '_: _' as the delimiter.
- The `Plan` and `Course` information are also contained in the _Plan Header_, 
  except here it is always labeled `Plan` never `Plan sum` even if the DVH is 
  for a Plan sum.
- Units for values are found at the end of the label in square brackets. e.g.
  > `Mean Dose [cGy]: 505.1`
- Unis will depend on the DVH style Chosen:
  
    <Table>
    <tr><th>Measurement Type</th><th>Absolute</th><th>Relative</th></tr>
    <tr><th>Dose</th><td>[cGy]</td><td>[%]</td></tr>
    <tr><th>Volume</th><td>[cm³]</td><td>[%]</td></tr>
    </Table>

- Some labels will not contain corresponding values. e.g.:
  > `Paddick CI: `
- Some labels will not contain an `N/A` value e.g.:
  > `D98.0% [%]: N/A`
- Sometimes a line will contain only a delimiter with neither a label or a value:
  > `: `
- A blank line occurs between at the end of the _Structure Dose Summary_ and 
  the start of the _DVH Curve Data_



<style type="text/css">
    .tg {text-align:left;background-color:#e6ffff;
        font-family:Arial, sans-serif;font-size:14px;font-weight:bold;
        border-style: solid dotted solid dotted}
    .th {text-align:center;background-color:#38fff8;font-weight:bold;}
    .tc {text-align:left;background-color:#e0ebeb;
        font-family: Terminal, monospace;font-size:11px;font-weight:normal;}
</style>
<table>
    <thead><tr>
        <th class="th">Export File Entry</th>
        <th class="th">Value</th>
        <th class="th">Example</th>
        </tr></thead>
    <tbody>
        <tr><td colspan="3" class="tg">Structure Information</td></tr>
        <tr>
            <td>Structure:</td>
            <td>ID of the structure</td>
             <td class="tc">Structure: Cricoid</td>
            </tr>
    <tr>
        <td>Approval Status:</td>
        <td>One of:
            <ul>
                <li>Approved</li>
                <li>Reviewed</li>
                <li>Unapproved</li>
                <li>Rejected</li>
                </ul>
            </td>
        <td class="tc">Approval Status: Approved</td>
        </tr>
    <tr>
        <td>Plan:</td>
        <td>ID of the plan</td>
        <td class="tc">Plan: EARR</td>
        </tr>
    <tr>
        <td>Course:</td>
        <td>ID of the course</td>
        <td class="tc">Course: C2</td>
        </tr>
    <tr>
        <td>Volume [cm³]:</td>
        <td>Volume. Value shown when dose is absolute.</td>
        <td class="tc">Volume [cm³]: 30.3</td>
        </tr>
    <tr>
    <tr><td colspan="3" class="tg">Structure Dose Summary</td></tr>
    <tr>
        <td>Dose Cover. [%]:</td>
        <td>Percentage of the dose coverage</td>
        <td class="tc">Dose Cover.[%]: 100.0</td>
        </tr>
    <tr>
        <td>Sampling Cover. [%]:</td>
        <td>Percentage of the structure volume used in DVH calculation</td>
        <td class="tc">Sampling Cover.[%]: 100.0</td>
        </tr>
    <tr>
        <td>Min Dose [%]:/Min Dose [Gy]:</td>
        <td>Dose minimum in percentage (relative dose) or Gray (absolute dose)</td>
        <td class="tc">Min Dose [cGy]: 272.3</td>
        </tr>
    <tr>
        <td>Max Dose [%]:/Max Dose [Gy]:</td>
        <td>Dose maximum in percentage (relative dose) or Gray (absolute dose)</td>
        <td class="tc">Max Dose [%]: 106.9</td>
        </tr>
    <tr>
        <td>Mean Dose [%]:/Mean Dose [Gy]:</td>
        <td>Dose mean in percentage (relative dose) or Gray (absolute dose)</td>
        <td class="tc">Mean Dose [%]: 101.6</td>
        </tr>
    <tr>
        <td>Modal Dose [%]:/Modal Dose [Gy]:</td>
        <td>Dose modal in percentage (relative dose) or Gray (absolute dose)</td>
        <td class="tc">Modal Dose [%]: 101.5</td>
        </tr>
    <tr>
        <td>Median Dose [%]:/Median Dose [Gy]:</td>
        <td>Dose median in percentage (relative dose) or Gray (absolute dose)</td>
        <td class="tc">Median Dose [%]: 101.6</td>
        </tr>
    <tr>
        <td>STD [%]:</td>
        <td>Standard deviation</td>
        <td class="tc">STD [%]: 1.1</td>
        </tr>
    <tr>
        <td>NDR:</td>
        <td>Natural dose ratio</td>
        <td class="tc"></td>
        </tr>
    <tr>
        <td>Equiv. Sphere Diam. [cm]:</td>
        <td>Equivalent sphere diameter value</td>
        <td class="tc">Equiv. Sphere Diam. [cm]: 4.2</td>
        </tr>
    <tr>
        <td>Conformity Index:</td>
        <td>Conformity index value</td>
        <td class="tc">Conformity Index: 1.00</td>
        </tr>
    <tr>
        <td>Gradient Measure [cm]:</td>
        <td>Gradient measure value</td>
        <td class="tc">Gradient Measure [cm]: 0.76</td>
        </tr>
    <tr>
        <td>Dose Level [cGy]:</td>
        <td></td>
        <td class="tc">Dose Level [cGy]: </td>
        </tr>
    <tr>
        <td>RTOG CI:</td>
        <td>Radiation Therapy Oncology Group Conformity Index value</td>
        <td class="tc">RTOG CI: </td>
        </tr>
    <tr>
        <td>Paddick CI:</td>
        <td>Paddick Conformity Index value</td>
        <td class="tc">Paddick CI: </td>
        </tr>
    <tr>
        <td>GI:</td>
        <td>Gastrointestinal index value</td>
        <td class="tc">GI: </td>
        </tr>
    <tr>
        <td>ICRU83 HI:</td>
        <td>ICRU 83 Homogeneity Index value</td>
        <td class="tc">ICRU83 HI: </td>
        </tr>
    <tr>
        <td>d<sub>Volume</sub> <b>OR</b> d<sub>Dose</sub> [cm³ <b>OR</b> Gy]:</td>
        <td>Differential DVH curve values</td>
        <td class="tc">
            D95.0% [cGy]: 6400.0<br>
            V95.0% [cm³]: 38.3086
            </td>
        </tr>
    <tr>
        <td>d<sub>Volume</sub>  <b>OR</b> d<sub>U</sub> 
            [cm³ * Gy1.5] <b>OR</b> U(Dose) = Dose-1.5:</td>
        <td>Natural DVH curve values</td>
        <td class="tc"></td>
        </tr>
</tbody>
</table>

**Example _Structure Information_ and _Structure Dose Summary_**

> ```
> Structure: opt PTV
> Approval Status: Approved
> Plan: EARR
> Course: C2
> Volume [cm³]: 38.3
> Dose Cover.[%]: 100.0
> Sampling Cover.[%]: 100.0
> Min Dose [%]: 91.9
> Max Dose [%]: 106.9
> Mean Dose [%]: 101.6
> Modal Dose [%]: 101.5
> Median Dose [%]: 101.6
> STD [%]: 1.1
> Equiv. Sphere Diam. [cm]: 4.2
> Conformity Index: 1.00
> Gradient Measure [cm]: 0.76
> Dose Level [cGy]: 
> RTOG CI: 
> Paddick CI: 
> GI: 
> ICRU83 HI: 
> D95.0% [cGy]: 6400.0
> D98.0% [%]: 99.0
> D99.0% [cGy]: 6279.5
> V95.0% [cm³]: 38.3086
> V100.0% [cm³]: 36.4045
> : 
> 
> ```

#### Structure Section Notes
- Start the section with a line that begins with *Structure:.*
- End after the next blank line.
- Split each line at the first ':'
- Remove leading and trailing spaces from each part of the split
- Convert the split rows into dictionary items, with the first item as 
  the key and the second as the value, dropping any empty rows.
- Extract any units as a separate item and convert the value to 
  a number e.g.<br>
    `D95.0% [cGy]: 6400.0` becomes: 
    - 'D95.0%': 6400.0
    - 'D95.0% unit': 'cGy'


#### Custom Processing Functions
The Plan sections requires custom functions to reliably extract information in 
a convenient format. The custom functions are defined as **Rules**, which 
identify the appropriate line(s) and then apply custom formatting.

Since the desired assembled result for the section is a dictionary, and since 
the dictionary is conveniently created from a sequence of length-two lists, 
each rule returns (or yields) one or more length-two lists.

The **Rules** are then combined as a **RuleSet**, where the rules are applied 
in order until one of them passes and the result of that rule is returned.

A default Processing Function is given that is applied if none of the rules 
pass.  This default function also returns a length-two list by splitting the 
line at the first ':' and returning the two parts after stripping spaces.

In [16]:
# Prescribed Dose Rule
def make_dose_data_rule() -> Rule:
    '''Split dose parameter into label, value and unit.
    The line:
        Volume [cm³]: 38.3
    Results in:
        ['Volume', 38.3],
        ['Volume unit', 'cm³']
    '''
    def parse_dose_data(line, event) -> tp.ProcessedList:
        match_results = event.test_value.groupdict()
        # Convert numerical value to float 
        match_results['value'] = float(match_results['value'])
        value_label = match_results['label'].strip()
        unit_label = value_label + ' unit'
        parsed_lines = [
            [value_label, match_results['value']],
            [unit_label, match_results['unit']]
            ]
        for line in parsed_lines:
            yield line

    structure_dose_pattern = (
        r'^(?P<label>[^[]+)'   # Initial parameter label
        r'\['                  # Unit start delimiter '['
        r'(?P<unit>[^\]]+)'    # unit group: text surrounded by []
        r'\]'                  # Unit end delimiter ']'
        r'\s*:\s*'             # Value delimiter with possible whitespace
        r'(?P<value>[0-9.]+)'  # value group Number
        r'\s*'                 # drop trailing whitespace
        r'$'                   # end of string
        )
    re_pattern = re.compile(structure_dose_pattern)
    dose_rule = Rule(sentinel=re_pattern, name='make_dose_data_rule',
                        pass_method=parse_dose_data, fail_method='None')
    return dose_rule

#### Build the RuleSet from the above Rules

In [17]:
dose_rule_set = RuleSet([make_dose_data_rule()], default=plan_split)

#### Test for blank line
- This function is used to identify the break between the structure section and the DVH Curve Data

In [18]:
def is_blank(line: str):
    return len(line) == 0

### Structure Section Parameters
<style type="text/css">
    .tg {text-align:left;background-color:#e6ffff;font-family:Arial, sans-serif;font-size:14px;font-weight:bold;border-style: solid dotted solid dotted}
    .th {text-align:center;background-color:#38fff8;font-weight:bold;}
    .tc {text-align:left;font-family: Terminal, monospace;font-size:11px;font-weight:normal;}
</style>
<table>
<tr><td>name</td><td>Plan</td><td>Section name for reference</td></tr>
<tr><td>start_section</td><td>('Structure:', 'START', 'Before')</td>
    <td>Start the section just before the first line that begins with 
        <i>Structure:</i>
        </td>
    </tr>
<tr><td>end_section</td><td>(is_blank, None, 'Before')</td>
    <td>End the section before the first line where is_blank(*line*) 
        returns True.
        </td>
    </tr>
<tr><td>processor</td>
  <td>[dose_rule_set]</td>
  <td>If the line contains units convert it into two 2-item lists:
        <ul>
            <li class="tc">[Value Label, Value]</li>
            <li class="tc">[Unit Label, Unit]</li>
            </ul>
      Otherwise, split into one 2-item list:
        <ul>
            <li class="tc">[Label, Value]</li>
            </ul>
      </td>
  </tr>
  <tr><td>assemble</td><td>plan_lookup</td>
  <td>Convert the split rows into dictionary items, with the first item as 
      the key and the second as the value.</td>
  </tr>
  </table>

In [19]:
dose_info_section = Section(
    name='Structure',
    start_section=('Structure:', 'START', 'Before'),
    end_section=(is_blank, None, 'Before'),
    processor=[dose_rule_set],
    assemble=tp.to_dict
    )

In [20]:
demo_dvh_text = demo_dvh_1.read_text(encoding='utf_8_sig').splitlines()
dose_info_section.read(demo_dvh_text)

{'Structure': 'BODY',
 'Approval Status': 'Approved',
 'Plan': 'CHWR',
 'Course': 'C1',
 'Volume': 20449.1,
 'Volume unit': 'cm³',
 'Dose Cover.': 100.0,
 'Dose Cover. unit': '%',
 'Sampling Cover.': 100.0,
 'Sampling Cover. unit': '%',
 'Min Dose': 0.0,
 'Min Dose unit': '%',
 'Max Dose': 103.4,
 'Max Dose unit': '%',
 'Mean Dose': 6.2,
 'Mean Dose unit': '%',
 'Modal Dose': 0.0,
 'Modal Dose unit': '%',
 'Median Dose': 0.3,
 'Median Dose unit': '%',
 'STD': 20.3,
 'STD unit': '%',
 'Equiv. Sphere Diam.': 33.9,
 'Equiv. Sphere Diam. unit': 'cm',
 'Conformity Index': 'N/A',
 'Gradient Measure [cm]': 'N/A',
 'Dose Level [cGy]': '',
 'RTOG CI': '',
 'Paddick CI': '',
 'GI': '',
 'ICRU83 HI': '',
 'D95.0%': 0.1,
 'D95.0% unit': 'cGy',
 'D98.0%': 0.0,
 'D98.0% unit': '%',
 'D99.0%': 0.0,
 'D99.0% unit': 'cGy',
 'V95.0%': 400.9586,
 'V95.0% unit': 'cm³',
 'V100.0%': 93.2574,
 'V100.0% unit': 'cm³',
 '': ''}

### DVH Curve Section
- Columns containing DVH data for the structure occurs immediately after the 
  related Structure Information.
- The number of lines of data will depend on:
    - The selected resolution of the DVH (0.1 cGy step size is the default)
    - The maximum dose to any structure included in the DVH.
    - The minimum and maximum dose range selected for the DVH.
- Columns are formatted in right-justified, fixed-width style 
- The data is contained in two or three of the following columns:
    - Dose [cGy]
    - Relative dose [%]
    - Structure Volume [cm³]
    - Ratio of Total Structure Volume [%]
    - dVolume / dDose [cm³ / cGy]
- The right most header sometimes extends past the fixed-width column end.
- Plan sum DVH tables (where relative dose is not defined only have two columns, 
  and have shorter column widths.
    
<style type="text/css">
    th {text-align:center;background-color:#38fff8;font-weight:bold;}
    .tg {text-align:left;background-color:#e6ffff;
        font-family:Arial, sans-serif;font-size:14px;font-weight:bold;
        border-style: solid dotted solid dotted}
    .tc {text-align:left;background-color:#e0ebeb;
        font-family: Terminal, monospace;font-size:11px;font-weight:normal;}
    .tb {border-width: medium thin thin thin;}
</style>
<table>
    <thead><tr><th>Selected DVH units</th> <th>Data Columns</th></tr></thead>
    <tbody>
    <tr><td rowspan="3">
            Plan,<br>
            Relative Dose,<br>
            Relative Volume
            </td>
        <td>Relative dose [%]</td></tr>
    <tr><td>Dose [cGy]</td></tr>
    <tr><td>Ratio of Total Structure Volume [%]</td></tr>
    <tr><td rowspan="3">
            Plan,<br>
            Absolute Dose,<br>
            Relative Volume
            </td>
        <td>Dose [cGy]</td></tr>
    <tr><td>Relative dose [%]</td></tr>
    <tr><td>Ratio of Total Structure Volume [%]</td></tr>
    <tr><td rowspan="3">
            Plan,<br>
            Relative Dose,<br>
            Absolute Volume
            </td>
        <td>Relative dose [%]</td></tr>
    <tr><td>Dose [cGy]</td></tr>
    <tr><td>Structure Volume [cm³]</td></tr>
    <tr><td rowspan="3">
            Plan,<br>
            Absolute Dose,<br>
            Absolute Volume
            </td>
        <td>Dose [cGy]</td></tr>
    <tr><td>Relative dose [%]</td></tr>
    <tr><td>Structure Volume [cm³]</td></tr>
    <tr><td rowspan="3" class="tb">
            Plan, Differential DVH<br>
            Relative Dose,<br>
            Absolute Volume
            </td>
        <td class="tb">Relative dose [%]</td></tr>
    <tr><td>Dose [cGy]</td></tr>
    <tr><td>dVolume / dDose [cm³ / %]</td></tr>
    <tr><td rowspan="3">
            Plan, Differential DVH<br>
            Absolute Dose,<br>
            Absolute Volume
            </td>
        <td>Dose [cGy]</td></tr>
    <tr><td>Relative dose [%]</td></tr>
    <tr><td>dVolume / dDose [cm³ / cGy]</td></tr>
    <tr><td rowspan="2" class="tb">
            Plan sum,<br>
            Absolute Dose,<br>
            Relative Volume
            </td>
        <td class="tb">Dose [cGy]</td></tr>
    <tr><td>Ratio of Total Structure Volume [%]</td></tr>        
    <tr><td rowspan="2">
            Plan sum,<br>
            Absolute Dose,<br>
            Absolute Volume
            </td>
        <td>Dose [cGy]</td></tr>
    <tr><td>Structure Volume [cm³]</td></tr>        
    </table>
    </tbody>   
    
**Column Widths**
<table>
    <thead><tr>
        <th>DVH plan type</th><th>Column Widths</th>
        </tr></thead>
    <tbody>
    <tr>
        <td>Normal plan</td>
        <td>
            0 - 16<br>
            17 - 36<br>
            37 - end
            </td>
        </tr>
    <tr>
        <td>Plan Comparison</td>
        <td>
            0 - 16<br>
            17 - 36<br>
            37 - end
            </td>
        </tr>
    <tr>
        <td>Plan Sum</td>
        <td>
            0 - 9<br>
            10 - end
            </td>
        </tr>       
    </table>
    </tbody>   


**Example _DVH Curve Data_** (Numbers at the top are for reference only)

> ```
>00000000001111111111222222222233333333334444444444555555555566666666667777777777
>01234567890123456789012345678901234567890123456789012345678901234567890123456789
>
>       Dose [cGy]   Relative dose [%] Ratio of Total Structure Volume [%]
>                0                   0                       100
>                1            0.015625                       100
>                2             0.03125                       100
>                3            0.046875                       100
> ...
>             6841             106.891                         0
>             6842             106.906                         0
>             6843             106.922                         0
>             6844             106.938                         0
>
>```

 > ```
>00000000001111111111222222222233333333334444444444555555555566666666667777777777
>01234567890123456789012345678901234567890123456789012345678901234567890123456789
>
>Dose [cGy] Ratio of Total Structure Volume [%]
>         0                       100
>         1                   81.5558
>         2                   79.9258
>         3                   78.2589
>...
>      4693                         0
>      4694                         0
>      4695                         0
>      4696                         0
>      4697                         0
>
>```

### DVH Curve Notes

- Start the section on the first line containing 'Dose' after the 
  *Structure* section ends.
- End before the line containing *'Structure:'* 
- Split the first line (header) into two equal-length lists.
    - The first list contains the Label
    - The second contains the units.
    - Both lists should have equal length (2 or 3)
    - Split by looking for a patter like: `text<space>[text]`
- Split the remaining , by looking for ']'
- Split each line at the first ':'
- Remove leading and trailing spaces from each part of the split
- Convert the split rows into dictionary items, with the first item as 
  the key and the second as the value, dropping any empty rows.
- If *Plan sum*, convert key to *Plan*
- If *Plan sum*, Add context information indicating that it is a plan sum
- If *Plan Status* contains approval information, split this information into 
  three parts.  e.g.<br>
  `Treatment Approved Thursday, January 02, 2020 12:55:56 by gsal` becomes:
    - 'Plan Status': 'Treatment Approved'
    - 'Approved on': 'January 02, 2020 12:55:56'
    - 'Approved by':  'gsal'
- Extract the prescribe dose units as a separate item and convert the dose to 
  a number e.g.<br>
    `Total dose [cGy]: 6400.0` becomes: 
    - 'Prescribed dose': 6400.0
    - 'Prescribed dose unit': 'cGy'
- Convert the prescription isodose line to a number e.g.<br>
  `% for dose (%): 100.0` becomes: 
    - 'Prescribed isodose line': 100.0
- If *Total dose* or  *% for dose (%)* have values of: `not defined`, replace 
  it with an empty string.



Split the data on spaces